# [PBL] Corrective RAG Agent

**학습 목표**   
6일간의 학습 내용을 바탕으로, 실제 업무에 적용 가능한 AI Agent를 직접 설계하고, 구현하는 과정을 경험하는 것을 목표로 합니다.  
본 실습에서는 다양한 형태의 데이터 파일을 해석하고 질문에 답변하는 Corrective RAG Agent를 구축해보겠습니다.  

## 1. 환경 설정

구현에 필요한 모든 모듈을 import합니다.   
LangChain의 핵심 컴포넌트들, 문서 처리 도구, 벡터 데이터베이스, 임베딩 모델, 웹 검색 도구, 그리고 유사도 계산을 위한 scikit-learn을 포함합니다.

In [17]:
import os
from typing import TypedDict, List, Dict, Any, Literal
from datetime import datetime

# --- LangChain 및 LangGraph 핵심 모듈 ---
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import StateGraph, END

# --- 데이터 처리 및 RAG 관련 모듈 ---
from langchain_community.document_loaders import UnstructuredFileLoader, PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_tavily import TavilySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- 유틸리티 ---
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## 2. API 키 설정

In [18]:
import os
from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

## 3. 상태 관리

Corrective RAG의 워크플로우에서 사용할 상태 객체를 정의합니다.    
사용자 질문, 검색 결과, 답변, 재시도 횟수, 현재 쿼리, 관련성 점수, 검색 소스(DB/웹) 등 모든 필요한 정보를 포함합니다.

In [19]:
class CorrectiveAgentState(TypedDict):
    user_question: str
    search_results: List[Document]
    final_answer: str
    retry_count: int
    max_retries: int
    current_query: str
    relevance_score: float
    docs_are_relevant: bool
    search_source: str 

## 4. 핵심 함수
Corrective RAG의 핵심 로직을 담당하는 5개의 노드 함수들을 구현합니다:
- `retrieve_node`: DB 또는 웹에서 문서 검색
- `grade_node`: 임베딩 유사도 기반으로 검색 결과 품질 평가
- `generate_answer_node`: 검색 결과를 바탕으로 답변 생성
- `rewrite_query_node`: 품질이 낮을 때 쿼리 재작성
- `final_answer_node`: 최종 답변 결정

In [20]:
def retrieve_node(state: CorrectiveAgentState, config: RunnableConfig) -> Dict[str, Any]:
    """문서 검색 (DB 또는 웹)"""
    source = state.get("search_source", "db")
    print(f"{source.upper()} 검색")
    
    if source == "db":
        retriever = config["configurable"]["retriever"]
        search_results = retriever.get_relevant_documents(state["current_query"])
        print(f"   - {len(search_results)}개 DB 문서 검색 완료")
    else:  # web
        try:
            # search_tool = TavilySearchResults(max_results=5)
            search_tool = TavilySearch(k=5)
            web_results = search_tool.invoke(state["current_query"])
            
            search_results = []
            for result in web_results:
                doc = Document(
                    page_content=f"제목: {result.get('title', '')}\n내용: {result.get('content', '')}",
                    metadata={"source": "web", "url": result.get('url', '')}
                )
                search_results.append(doc)
            print(f"   - {len(search_results)}개 웹 검색 결과")
        except Exception as e:
            print(f"   - 웹 검색 실패: {e}")
            search_results = []
    
    return {"search_results": search_results}

def grade_node(state: CorrectiveAgentState, config: RunnableConfig) -> Dict[str, Any]:
    """문서 관련성 평가 (DB/웹 공통)"""
    source = state.get("search_source", "db")
    print(f"{source.upper()} 검색 결과 평가")
    
    embedding_model = config["configurable"]["embedding_model"]
    
    if not state["search_results"]:
        return {
            "relevance_score": 0.0,
            "docs_are_relevant": False
        }
    
    # 질문 임베딩
    question_embedding = embedding_model.embed_query(state["user_question"])
    
    # 문서들 임베딩 및 유사도 계산
    similarities = []
    for doc in state["search_results"]:
        doc_embedding = embedding_model.embed_query(doc.page_content)
        similarity = cosine_similarity([question_embedding], [doc_embedding])[0][0]
        similarities.append(similarity)
    
    # 평균 유사도
    avg_similarity = np.mean(similarities)
    docs_are_relevant = avg_similarity >= 0.5
    
    print(f"   평균 유사도: {avg_similarity:.3f}")
    print(f"   관련성 판정: {'관련' if docs_are_relevant else '비관련'}")
    
    return {
        "relevance_score": avg_similarity,
        "docs_are_relevant": docs_are_relevant
    }

def generate_answer_node(state: CorrectiveAgentState, config: RunnableConfig) -> Dict[str, Any]:
    """답변 생성"""
    source = state.get("search_source", "db")
    print(f"답변 생성 ({source} 소스)")
    
    llm = ChatOpenAI(model=config["configurable"]["model_name"], temperature=0)
    
    context = "\n\n".join([doc.page_content for doc in state["search_results"]])
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "검색 결과를 기반으로 질문에 답변하세요."),
        ("human", "검색 결과:\n{context}\n\n질문: {question}\n\n답변:")
    ])
    
    response = (prompt | llm).invoke({
        "context": context,
        "question": state["user_question"]
    })
    
    return {"final_answer": response.content}

def rewrite_query_node(state: CorrectiveAgentState, config: RunnableConfig) -> Dict[str, Any]:
    """쿼리 재작성"""
    print("쿼리 재작성")
    
    llm = ChatOpenAI(model=config["configurable"]["model_name"], temperature=0)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "검색 결과가 부족한 경우, 더 구체적이고 명확한 검색어로 재작성하세요."),
        ("human", "원래 질문: {question}\n\n재작성된 검색어:")
    ])
    
    response = (prompt | llm).invoke({"question": state["user_question"]})
    print(f"재작성된 쿼리: {response.content}")
    
    return {
        "current_query": response.content,
        "retry_count": state["retry_count"] + 1,
        "search_source": "web"  # 재작성 후에는 웹 검색
    }

def final_answer_node(state: CorrectiveAgentState, config: RunnableConfig) -> Dict[str, Any]:
    """최종 답변"""
    if state["retry_count"] >= state["max_retries"]:
        return {"final_answer": "죄송합니다. 충분한 정보를 찾을 수 없어 답변을 생성할 수 없습니다."}
    else:
        return {"final_answer": state["final_answer"]}

## 5. Corrective RAG Agent 클래스
전체 Corrective RAG 시스템을 캡슐화한 메인 클래스입니다.    
워크플로우 구성, 문서 설정, 질의응답 기능을 제공하며, HuggingFace 임베딩 모델을 사용하여 벡터 검색을 수행합니다.

In [21]:
class CorrectiveRAG:
    def __init__(self, model_name="gpt-4o-mini"):
        self.model_name = model_name
        self.retriever = None
        
        # 임베딩 모델
        self.embedding_model = OpenAIEmbeddings(
            model="text-embedding-3-small"
        )
        
        self.workflow = self._build_workflow()
    
    def _build_workflow(self):
        """Corrective RAG 워크플로우"""
        workflow = StateGraph(CorrectiveAgentState)
        
        # 노드 추가
        workflow.add_node("retrieve", retrieve_node)
        workflow.add_node("grade", grade_node)
        workflow.add_node("generate", generate_answer_node)
        workflow.add_node("rewrite", rewrite_query_node)
        workflow.add_node("final_answer", final_answer_node)
        
        # 시작점 설정
        workflow.set_entry_point("retrieve")
        
        # 기본 흐름
        workflow.add_edge("retrieve", "grade")
        
        # 평가 후 분기
        def should_retry(state):
            if state["retry_count"] >= state["max_retries"]:
                print(f"   - 최대 시도 횟수 도달 ({state['max_retries']}회)")
                return "final_answer"
            
            if not state["docs_are_relevant"]:
                print(f"   - 재시도 필요: 유사도 점수 낮음 ({state['relevance_score']:.3f} < 0.5)")
                return "rewrite"
            else:
                print(f"   - 품질 통과: 유사도 점수 충분 ({state['relevance_score']:.3f} >= 0.5)")
                return "generate"
        
        workflow.add_conditional_edges(
            "grade",
            should_retry,
            {
                "rewrite": "rewrite",
                "generate": "generate",
                "final_answer": "final_answer"
            }
        )
        
        # 나머지 엣지
        workflow.add_edge("generate", "final_answer")
        workflow.add_edge("rewrite", "retrieve")
        workflow.add_edge("final_answer", END)
        
        return workflow.compile()
    
    def setup_documents(self, input_files: List[str]):
        """문서 초기 설정"""
        print("문서 설정 시작")
        
        # 문서 로딩
        all_docs = []
        for file_path in input_files:
            try:
                print(f"   로딩 중: {os.path.basename(file_path)}")
                
                if file_path.endswith('.pdf'):
                    loader = PyMuPDFLoader(file_path)
                else:
                    loader = UnstructuredFileLoader(file_path, mode="elements")
                
                docs = loader.load()
                all_docs.extend(docs)
                print(f"   - 로드 성공: {len(docs)}개 문서")
                
            except Exception as e:
                print(f"   - 로드 실패: {e}")
        
        # 문서 청킹
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        chunked_docs = text_splitter.split_documents(all_docs)
        
        # 메타데이터 제거
        clean_docs = []
        for doc in chunked_docs:
            clean_doc = Document(
                page_content=doc.page_content,
                metadata={}
            )
            clean_docs.append(clean_doc)
        
        # 벡터 DB 생성
        print("벡터 DB 생성 중...")
        vectorstore = Chroma.from_documents(
            documents=clean_docs,
            embedding=self.embedding_model
        )
        self.retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
        
        print("설정 완료")
    
    def ask(self, question: str) -> str:
        """질문에 답변"""
        initial_state = {
            "user_question": question,
            "search_results": [],
            "final_answer": "",
            "retry_count": 0,
            "max_retries": 2,
            "current_query": question,
            "relevance_score": 0.0,
            "docs_are_relevant": True,
            "search_source": "db"  # 처음에는 DB 검색
        }
        
        config = {
            "configurable": {
                "model_name": self.model_name,
                "retriever": self.retriever,
                "embedding_model": self.embedding_model
            }
        }
        
        print(f"\n질문: {question}")
        print("=" * 60)
        
        final_state = self.workflow.invoke(initial_state, config=config)
        
        print("=" * 60)
        print("최종 답변:")
        
        return final_state["final_answer"]

## 6. 에이전트 실행 및 테스트

In [22]:
# 파일 경로 설정
dataset_path = "./data/"
filenames = [
    "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks (2020, Facebook AI).pdf",
    "서울시_산학연_협력사업_운영요령_개정_전문.pdf",
    "피싱_메일_공격_사례_분석_및_대응_방안.pdf",
    "[이론1] Advanced RAG 개요.pptx",
    "README.md"
]
files_to_process = [os.path.join(dataset_path, f) for f in filenames]

# 에이전트 생성 및 설정
agent = CorrectiveRAG()
agent.setup_documents(files_to_process)

문서 설정 시작
   로딩 중: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks (2020, Facebook AI).pdf
   - 로드 성공: 19개 문서
   로딩 중: 서울시_산학연_협력사업_운영요령_개정_전문.pdf
   - 로드 성공: 27개 문서
   로딩 중: 피싱_메일_공격_사례_분석_및_대응_방안.pdf
   - 로드 성공: 40개 문서
   로딩 중: [이론1] Advanced RAG 개요.pptx
   - 로드 실패: partition_pptx() is not available because one or more dependencies are not installed. Use: pip install "unstructured[pptx]" (including quotes) to install the required dependencies
   로딩 중: README.md
   - 로드 실패: partition_md() is not available because one or more dependencies are not installed. Use: pip install "unstructured[md]" (including quotes) to install the required dependencies
벡터 DB 생성 중...
설정 완료


## 7. 테스트 예제
구현된 Corrective RAG Agent의 실제 사용을 테스트해 봅니다.     
시스템이 어떻게 DB 검색과 웹 검색을 자동으로 조합하여 답변을 생성하는지 확인할 수 있습니다.  

In [23]:
# 예시 1: DB에 관련 내용이 있는 질문
answer_text = agent.ask(question="피싱 메일 공격에 대한 기술적 대응 방안을 설명해줘.")
print(answer_text)


질문: 피싱 메일 공격에 대한 기술적 대응 방안을 설명해줘.
DB 검색
   - 5개 DB 문서 검색 완료
DB 검색 결과 평가
   평균 유사도: 0.661
   관련성 판정: 관련
   - 품질 통과: 유사도 점수 충분 (0.661 >= 0.5)
답변 생성 (db 소스)
최종 답변:
피싱 메일 공격에 대한 기술적 대응 방안은 다음과 같습니다:

1. **보안메일 사용**: 기업 및 기관에서는 보안메일을 도입하여 문서 보안을 강화할 수 있습니다. 보안메일은 특정 비밀번호를 입력해야만 열람할 수 있도록 설계되어 있어, 공격자가 이를 위장하여 피싱 메일을 유포하는 것을 방지할 수 있습니다. 그러나 보안메일을 위장한 피싱 메일에 주의해야 합니다.

2. **메일 열람 페이지 취약점 방지**: 메일 본문에 악성 스크립트를 삽입하여 사용자가 메일을 열람하는 것만으로도 피싱 사이트로 연결되는 경우가 있습니다. 이를 방지하기 위해 메일 클라이언트와 웹 메일 서비스의 보안 취약점을 지속적으로 점검하고 패치해야 합니다.

3. **사용자 교육**: 직원들에게 피싱 메일의 특징과 위험성을 교육하여, 의심스러운 메일을 클릭하지 않도록 하는 것이 중요합니다. 사용자가 링크를 클릭하거나 첨부파일을 열지 않도록 경각심을 높이는 것이 필요합니다.

4. **스팸 필터링 및 보안 솔루션 도입**: 스팸 필터링 솔루션을 도입하여 의심스러운 메일을 사전에 차단할 수 있습니다. 또한, 이메일 보안 솔루션을 통해 악성 코드 및 피싱 공격을 탐지하고 차단하는 기능을 강화해야 합니다.

5. **정기적인 보안 점검 및 업데이트**: 시스템과 소프트웨어의 보안 점검을 정기적으로 실시하고, 최신 보안 패치를 적용하여 취약점을 최소화해야 합니다.

6. **피싱 페이지 모니터링**: 피싱 페이지의 변화를 모니터링하고, 이를 통해 새로운 공격 기법을 파악하여 대응 방안을 마련하는 것이 중요합니다.

이러한 대응 방안을 통해 피싱 메일 공격의 위험을 줄이고, 조직의 정보 보안을 강화할 수 있습니다.


In [24]:
# 예시 2: DB에 관련 내용이 없는 질문
answer_text = agent.ask(question="최근 출시된 GPT-5에 대한 정보 알려줘")
print(answer_text)


질문: 최근 출시된 GPT-5에 대한 정보 알려줘
DB 검색
   - 5개 DB 문서 검색 완료
DB 검색 결과 평가
   평균 유사도: 0.336
   관련성 판정: 비관련
   - 재시도 필요: 유사도 점수 낮음 (0.336 < 0.5)
쿼리 재작성
재작성된 쿼리: 최근 출시된 GPT-5에 대한 정보와 특징, 기능, 사용 사례 등을 알려주세요.
WEB 검색
   - 웹 검색 실패: 'str' object has no attribute 'get'
WEB 검색 결과 평가
   - 재시도 필요: 유사도 점수 낮음 (0.000 < 0.5)
쿼리 재작성
재작성된 쿼리: 최근 출시된 GPT-5에 대한 정보와 특징, 기능, 사용 사례 등을 알려주세요.
WEB 검색
   - 웹 검색 실패: 'str' object has no attribute 'get'
WEB 검색 결과 평가
   - 최대 시도 횟수 도달 (2회)
최종 답변:
죄송합니다. 충분한 정보를 찾을 수 없어 답변을 생성할 수 없습니다.


## 8. RAGAS를 활용한 정량적 평가

4일차에 학습했던 RAGAS를 이용하여 제작한 RAG Agent를 평가해봅니다.

In [25]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [26]:
eval_data = {
    "question": [
        "서울시 산학연 협력사업의 세부사항 규정 관점에서 운영요령의 목적은 무엇인가요?",
        "운영요령에서 ‘주관연구개발기관’은 어떻게 정의되나요?",
        "운영요령이 규정한 지원 범위를 두세 가지 예로 들면 무엇인가요?",
        "운영요령 기준으로 자유공모와 지정공모의 차이는 무엇인가요?",
        "운영요령에 따른 협약 체결 기한은 어떻게 되나요?",
        "운영요령의 기관부담연구개발비 원칙은 무엇인가요?",
        "RCMS와 PMS는 각각 무엇을 하는 시스템인가요?",
        "연차보고서와 최종보고서는 언제 제출하나요?"
    ],
    "ground_truth": [
        "서울시 산학연 협력사업을 효율적으로 추진하기 위한 세부사항을 규정한다.",
        "주관연구개발기관은 연구개발과제를 총괄 수행하는 서울시 소재 기관이다.",
        "연구개발 지원, 기업 R&D 지원, 기술사업화·실증지원 등이 포함된다.",
        "자유공모는 수행기관이 주제를 자유롭게 제안하고, 지정공모는 과제를 미리 지정한 뒤 기관만 공모로 선정한다.",
        "선정 통보 후 20일 이내 서류를 제출하고 30일 이내 협약을 체결한다.",
        "기관부담은 현금과 현물로 구성하며 현금은 원칙적으로 총 부담금의 10% 이상이다.",
        "RCMS는 연구개발비 집행·정산 실시간 관리, PMS는 과제 정보·평가·관리를 위한 시스템이다.",
        "연차보고서는 매 연차 종료일까지, 최종보고서는 전체 기간 종료 후 2개월 이내 제출한다."
    ]
}


eval_df = pd.DataFrame(eval_data)

In [27]:
# Corrective RAG 시스템 평가
print("--- Corrective RAG 시스템 평가 시작 ---")

corrective_results = []
for i, row in eval_df.iterrows():
    question = row["question"]
    
    # Corrective RAG의 답변 생성
    answer = agent.ask(question)
    
    # 검색된 문서들 가져오기
    retrieved_docs = agent.retriever.get_relevant_documents(question)
    contexts = [doc.page_content for doc in retrieved_docs]
    
    corrective_results.append({
        "question": question,
        "answer": answer,
        "contexts": contexts,
        "ground_truth": row["ground_truth"]
    })

corrective_dataset = Dataset.from_list(corrective_results)

# RAGAS 평가 실행
corrective_metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision
]

corrective_eval_result = evaluate(
    dataset=corrective_dataset,
    metrics=corrective_metrics,
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small")
)

print("\n--- Corrective RAG 평가 결과 ---")
print(corrective_eval_result)
print(corrective_eval_result.to_pandas())

--- Corrective RAG 시스템 평가 시작 ---

질문: 서울시 산학연 협력사업의 세부사항 규정 관점에서 운영요령의 목적은 무엇인가요?
DB 검색
   - 5개 DB 문서 검색 완료
DB 검색 결과 평가
   평균 유사도: 0.554
   관련성 판정: 관련
   - 품질 통과: 유사도 점수 충분 (0.554 >= 0.5)
답변 생성 (db 소스)
최종 답변:

질문: 운영요령에서 ‘주관연구개발기관’은 어떻게 정의되나요?
DB 검색
   - 5개 DB 문서 검색 완료
DB 검색 결과 평가
   평균 유사도: 0.624
   관련성 판정: 관련
   - 품질 통과: 유사도 점수 충분 (0.624 >= 0.5)
답변 생성 (db 소스)
최종 답변:

질문: 운영요령이 규정한 지원 범위를 두세 가지 예로 들면 무엇인가요?
DB 검색
   - 5개 DB 문서 검색 완료
DB 검색 결과 평가
   평균 유사도: 0.349
   관련성 판정: 비관련
   - 재시도 필요: 유사도 점수 낮음 (0.349 < 0.5)
쿼리 재작성
재작성된 쿼리: 운영요령에서 규정한 지원 범위의 예시를 두세 가지로 설명해 주세요.
WEB 검색
   - 웹 검색 실패: 'str' object has no attribute 'get'
WEB 검색 결과 평가
   - 재시도 필요: 유사도 점수 낮음 (0.000 < 0.5)
쿼리 재작성
재작성된 쿼리: 운영요령에서 규정한 지원 범위의 예시 몇 가지는 무엇인가요?
WEB 검색
   - 웹 검색 실패: 'str' object has no attribute 'get'
WEB 검색 결과 평가
   - 최대 시도 횟수 도달 (2회)
최종 답변:

질문: 운영요령 기준으로 자유공모와 지정공모의 차이는 무엇인가요?
DB 검색
   - 5개 DB 문서 검색 완료
DB 검색 결과 평가
   평균 유사도: 0.371
   관련성 판정: 비관련
   - 재시도 필요: 유사도 점수 낮음 (0.371 < 0.5)
쿼리 재작성
재작성된 쿼리: 운영요령 기준 자유

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]


--- Corrective RAG 평가 결과 ---
{'faithfulness': 0.3417, 'answer_relevancy': 0.2193, 'context_recall': 0.6250, 'context_precision': 0.7111}
                                    user_input  \
0  서울시 산학연 협력사업의 세부사항 규정 관점에서 운영요령의 목적은 무엇인가요?   
1                운영요령에서 ‘주관연구개발기관’은 어떻게 정의되나요?   
2          운영요령이 규정한 지원 범위를 두세 가지 예로 들면 무엇인가요?   
3             운영요령 기준으로 자유공모와 지정공모의 차이는 무엇인가요?   
4                  운영요령에 따른 협약 체결 기한은 어떻게 되나요?   
5                   운영요령의 기관부담연구개발비 원칙은 무엇인가요?   
6                 RCMS와 PMS는 각각 무엇을 하는 시스템인가요?   
7                      연차보고서와 최종보고서는 언제 제출하나요?   

                                  retrieved_contexts  \
0  [서울시산학연협력사업운영요령\n서울특별시\n- 1 / 27 -\n서울시 산학연 협력...   
1  [마. 그밖의관리지침, 세부사업별공고등으로정하는기관·단체\n3. “주관연구개발기관”...   
2  [서울시산학연협력사업운영요령\n서울특별시\n- 5 / 27 -\n2. 기술분야별기술...   
3  [서울시산학연협력사업운영요령\n서울특별시\n- 14 / 27 -\n2. 연구개발기관...   
4  [서울시산학연협력사업운영요령\n서울특별시\n- 27 / 27 -\n제1조(시행일) ...   
5  [에대한참여를제한하거나이미지급한서울시연구개발비의5배의범위에서제재부\n가금을부과할수있...   
6  [연구개발비사용기준을위반한행위

In [28]:
print("\n--- Corrective RAG 시스템 성능 확인인 ---")
eval_df_result = corrective_eval_result.to_pandas()

metrics_summary = {
    'faithfulness': eval_df_result['faithfulness'].mean(),
    'answer_relevancy': eval_df_result['answer_relevancy'].mean(),
    'context_recall': eval_df_result['context_recall'].mean(),
    'context_precision': eval_df_result['context_precision'].mean()
}

df = pd.DataFrame(metrics_summary, index=[0])

print(df)


--- Corrective RAG 시스템 성능 확인인 ---
   faithfulness  answer_relevancy  context_recall  context_precision
0      0.341667           0.21925           0.625           0.711111
